In [ ]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML 

# Введение в теорию или как связаны подгузники и пиво?

   Обучение на ассоциативных правилах (далее Assocoations rules learning - ARL) представляет из себя с одной стороны, простой, с другой - довольно часто применимый в реальной жизни метод поиска взаимосвязей (ассоциаций) в датасетах или, если точнее, айтемсетах (itemsests). Именно ARL лежит в основе многих рекомендательных систем, работающих в интернет-магазинах. 
   Впервые подробно об этом заговорил Piatesky-Shapiro G [1] в работе “Discovery, Analysis, and Presentation of Strong Rules.” (1991) Более подробо тему развивали Agrawal R, Imielinski T, Swami A в работах “Mining Association Rules between Sets of Items in Large Databases” (1993) [2] и “Fast Algorithms for Mining Association Rules.” (1994) [3].

    
   


   
  В общем виде ARL можно опсиать как "Who bought x also bought y" ("Кто купил x, также купил y"). В основе лежит анализ транзакций (transactions), внутри каждой из которых лежит свой уникальный itemset из набора items. При помощи ARL алогритмов нахоядтся те самые "правила" совпаденяи items внутри одной транзакции, котоыре потом сортируются по их силе. 

   Да, вот все так просто и логично.

   За этой простотой, однако, могут скрываться поразительные вещи, о которых common sense даже не подозревал:) 

   Классический случай такого когнитивного диссонанса описан в статье D.J. Power "Ask Dan!", опубликованной в DSSResources.com [4]: 

   В 1992 году, группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (нет, там продавали не наркотики и даже не лекарства, точнее, не только лекартсва. Drug Store в стране веротяного противника - формат разнокалиберных магазинов у дома). 
   После анализа всех этих транзакций самым сильным правилом получилось "Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе". 
   К сожалению такое правило показалось руководству Osco Drug настолкьо контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали:) Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с рабоыт домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайшей магазин. И мужья, не долго думая, совмещали приятное с полезным - покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения

# Описание Association rule

Итак мы выяснили, что пиво и подгузники - хороший джентельменский набор, а что дальше? 

Пусть у нас имеется некий датасет (или коллекция) D, такой, что D = {d_0 ... d_j}, где d - уникальная транзакция-itemset (например, кассовый чек).
Внутри каждой d представлен набор items (i - item), причем в идеально случае он представлен в бинарном виде: d1 = [{Пиво: 1}, {Вода: 0}, {Кола: 1}, {...}], d2 = [{Пиво : 0}, {Вода: 1}, {Кола : 1}, {...}]. Принято каждый itemset описывать через количество ненулевых значений (k-itemset), например, [{Пиво: 1}, {Вода: 0}, {Кола: 1}] являестя 2-itemset.

Если в изначальном виде не представлен, можно при желании руками его преобразовать (OHE и pd.get_dummies вам в помощь).

Таким образом, датасет представляет собой разреженную матрицу со значениями {1,0}. Это будет бинарный датасет. Существуют и другие виды записи - вертикальный датасет и транзакционный датасет

ОК, данные преобразовали, как найти правила?

Существует целый ряд ключевых (если хотите - базовых) понятий в ARL, которые нам помогут эти правила вывести.

##### Support (поддержка)

Первое понятие  в ARL - support:


$supp(X) = \frac{\{t\in T;\ X \in t\}}{|T|}$

, где X - itemset, содержащий в себе i-items, а T - количество транзакций. Т.е. в общем виде это показатель "частотности" данного itemset во всех анализируемых транзакциях. Но это касается только X. Нам же интересен скорее вариант, когда у нас в одном itemset встречаются x1 и x2 (например).
Ну тут тоже все просто. Пусть x1 = {Пиво}, а x2 = {Подгузники}, значит нам нужно посчитать, во скольких транзакициях втречаестя эта парочка. 

$supp(x_1\cup x_2) = \frac{\sigma(x_1 \cup x_2)}{|T|}$, где $\sigma $ - количество транзакций, содержащих $x_1$ и $x_2$

##### Confidence (уровень доверия)

Следующее ключевое понятие - confidence. Это показатель того, как часто наше правило срабатывает для всего датасета. 

$conf(x_1\cup x_2) = \frac{supp(x_1 \cup x_2)}{supp(x_1)}$

Приведем пример: мы хотим посчитать confidence для правила "кто покупает пиво, тот покупает и подгузники".

Для этого сначала посчитаем, какой support у правила "покупает пиво", потом посчитаем support у правила "покупает пиво и подгузники", и просто поделим одно на другое. Т.е. мы посчитаем в скольких случаях (транзакциях) срабатывает правило "купил пиво $supp(X)$, купил подгузники и пиво $supp(X \cup Y)$"
Ничего не напоминает? Байес смотрит на все это несколько недоуменно и с презрением:)

<img src="src/images/thomas-bayes.png">

##### Lift (подъем или повышение)

Следующее понятие в нашем списке - lift. Грубо говоря, lift - это отношение "зависимости" items к их "независимости". Lift показывает, насколько items зависят друг от друга. Это очевидно из формулы:

$lift(x_1\cup x_2) = \frac{supp(x_1 \cup x_2)}{supp(x_1) \times supp(x_2)}$

Например, мы хотим понять зависимость покупки пива и покупки подгузников. Для этого считаем support правила "купил пиво и подгузники" и делим его на произведение правил "купил пиво" и "купил подгузники". В случае, если lift = 1, мы говорим, что items независимы и правил совместной покупки тут нет. Если же lift > 1, то велечина, на которую lift, собственно, больше этой самой единицы, и покажет нам "силу" правила. Чем больше единицы, тем круче.

##### Conviction (убедительность)

В общем виде Conviction - это "частотность ошибок" нашего правила. Т.е., например, как часто покупали пиво без подгузников и наоборот.

$conv(x_1\cup x_2) = \frac{1 - supp(x_2)}{1 - conf(x_1 \cup x_2)}$

Чем результат по формуле выше ближе к 1, тем лучше. Например, если conviction покупки пива и подгузников вместе равен 1.2, это значит, что правило "купил пиво и подгузники" было бы в 1.2 раза (на 20%) более верным, чем если бы совпадение этих items в одной транзакции было бы чисто случайным. Немного неинтуитивное понятие, но оно и используестя н етак часто, как предыдущие три. 

###### <<<<<Отсутпление про брутфорс>>>>>>

### Apriori Algorithm

#### Теория

Используемые понятия:
<br> -  Множество объектов (**itemset**): $X \subseteq I = \{x_1, x_2, ..., x_n\}$</br>
<br> -  Множество идентификаторов транзакций (**tidset**): $T = \{t_1, t_2, ..., t_m\}$</br>
<br> -  Множество транзакций (**transactions**): $\{(t,\ X):\ t\in T,\ X \in I\}$</br>

Сложность brute-force алгоритма:
Для того чтобы найти все возможные Association rules применяя brute-force алгоритм нам необходимо перечислить все подмножества $X$ из набора $I$ и для каждого подмножества $X$ рассчитать $supp(X)$. Данный подход будет состоять из следующих шагов:
-  генерация **кандидатов**. Данный шаг состоит из перебора всех возможных подмножеств $X$ подмножества $I$. Данные подмножества называются **кандидатами**. Поскольку каждое подмножество теоретически может являться часто встречаемым подмножеством, то множество потенциальных кандидатов будет состоять из $2^{|I|}$ элементов (здесь $|I|$ обозначается число элементов множества $I$, а $2^{|I|}$ часто называется булеаном множества $I$, то есть множество всех подмножеств $I$)
-  расчет **support**. На данном шаге рассчитывается $supp(X)$ каждого кандидата $X$

Таким образом, сложность нашего алгоритма будет: $O(|I|*|D|*2^{|I|})$, где
<br>$O(2^{|I|})$ - количество возможных кандидатов</br>
<br>$O(|I|*|D|)$ - сложность расчета sup(X). Поскольку для расчета  $supp(X)$ нам необходимо перебрать все элементы в $I$ в каждой транзакции $t \in T$</br>

Введем дополнительно еще несколько понятий.
<br>Будем рассматривать дерево префиксов (prefix tree), где 2 элемента $X$ и $Y$ соединены, если $X$ является прямым подмножеством $Y$. Таким образом мы можем пронумеровать все подмножества множества $I$. Рисунок приведен ниже</br>

<img src="/src/images/tree_1.png">

Итак, рассмотрим Apriori алгоритм.
<br>
Apriori алгоритм использует следующее утверждение: если $X \subseteq Y$, то $supp(X) \geq supp(Y)$. Откуда следуют следующие 2 свойства:
 -  если $Y$ встречается часто, то любое подмножество $X: X \subseteq Y$  так же встречается часто
 -  если $X$ встречается редко, то любое супермножество $Y: Y \supseteq X$ так же встречается редко
 </br>

Apriory алгоритм по-уровнево проходит по префиксному дереву и рассчитывает частоту встрчаемости подмножеств $X$ в $D$. Таким образом, в соответствии с алгоритмом:
 -  исключаются редкие подмножества и все их супемножества
 -  рассчитывается $supp(X)$ для кадого подходящего кандидата $X$ размера $k$ на уровне $k$

<img src="/src/images/tree_2.png">

#### Реализация в Python

from sklearn import... эммм... а импортировать-то и нечего:) На данный момент модулей для ALR в sklearn нет. Ну ничего, погуглим или напишем свои, правда?

По сети гуляет целый ряд реализаций, например вот - https://github.com/asaini/Apriori, вот - http://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/, и даже вот -https://codereview.stackexchange.com/questions/38101/apriori-algorithm-using-pandas

Мы же на практике придерживаемся алгоритма apyori, написанного Ю Мочиузки (Yu Mochizuki). Полный код приводить не будем, желающие могут посмотреть тут - https://github.com/ymoch/apyori , а вот архитектуру решения и пример использования покажем.

Условно решение Мочизуки можно разделить на 4 части: Структура данных, Внутренние функции, API и Прикладные функции.

Первая часть модуля (Структура данных) работает с изначальным датасетом. Реализуется класс TransactionManager, методы которого объединяют транзакции в матрицу, формируют список правил-кандидатов и считают support для каждого правила. Внутренние функции дополнительно по support'у формируют списки правил и соответственно их ранжируют. API логично позволяет работать напрямую с датасетами, а Прикладные функции позволяют обрабатывать транзакции и выводить результат в читаемый вид. Никакого rocketscience.  

Посмотрим, как использовать модуль на реальном (ну, в данном случае - игрушечном) датасете.

In [7]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# загрузим данные
dataset = pd.read_csv('src/data/Market_Basket_Optimisation.csv', header = None)

In [9]:
#dataset.describe #всего 7500 записей-транзакций

In [10]:
# посомтрим на датасет
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Видим, что датасет у нас представляет разреженную матрицу, где в строках у нас набор items в каждой транзакции.

In [ ]:
#создаим из них матрицу
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

In [ ]:
#загружаем apriori
import apyori

In [ ]:
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получит
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2))

In [ ]:
# вот с визуализацией в пакете проблемы, точнее геморрой....

In [ ]:
import shutil, os

In [ ]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [ ]:
import json #преобразовывать будем в json, используя встроенные в модуль методы

In [ ]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

In [ ]:
# и взгялнем на итоги
pd.set_option('display.max_colwidth', -1)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

In [6]:
data.head()

NameError: name 'data' is not defined

Итого мы видим:

1. Пары items
2. items_base - первый элемент пары
3. items_add - второй (добавленный алгоритмом) элемент пары
4. confidence - значение confidence для пары
5. lift - значение lift для пары
6. support - начение support для пары. При желании, по нему можно отсортировать 


Результаты логичные: эскалоп и макароны, эскалоп и сливочно-грибной соус, курица и нежирная сметана, мягкий сыр и мед и т.д. - все это вполне логичные и, главное, вкусные сочетания:)

#### Реализация в R

ARL тот случай,скогда R-фаги могут злорадно похихикать. В R реализована библиотека arules, где присутствует и apriori, и другие алгоритмы. Официальную доку можно посмотреть тут - https://cran.r-project.org/web/packages/arules/arules.pdf

Посмотрим на нее в действии:

Для начала установим ее (если еще не установили):
<br>
install.packages('arules')

Считаем данные и преобразуем их в матрицу транзакций:
<BR>
<br>
$library(arules)$
<br>
$dataset = read.csv('Market_Basket_Optimisation.csv', header = FALSE)$
<br>
$dataset = read.transactions('Market_Basket_Optimisation.csv', sep = ',', rm.duplicates = TRUE)$

Посмотрим на данные:
<BR>
$summary(dataset)$
<br>
$itemFrequencyPlot(dataset, topN = 10)$

Выучим наши правила:
<br>
В общем виде фнкция вызова apriori выглядит так:  $apriori(data, parameter = NULL, appearance = NULL, control = NULL)$, где
<br>

$data$ - наш датасет
<br>
$paramter$ - список (list) параметров для модели: минимальные support, confidence и lift
<br>
$appearance$ - отвечает за отображение данных. Может принимать значения lhs, rhs, both, items, none, которые определяют положение items в output
<br>
$control$ - отвечает за сортировку вывода (ascending, descending, без сортировки), а также за то, отображать ли прогрессбар или нет (параметр verbose)



Обучим модель:

$rules = apriori(data = dataset, parameter = list(support = 0.004, confidence = 0.2))$

И посмотрим на результаты:
<br>
$inspect(sort(rules, by = 'lift')[1:10])$

Убедимся, что на выходе имеем примерно те же результаты, что при использовании модуля apyori в Python:
<br>

1. {light cream}                               => {chicken}       0.004532729
2. {pasta}                                     => {escalope}      0.005865885
3. {pasta}                                     => {shrimp}        0.005065991
4. {eggs,ground beef}                          => {herb & pepper} 0.004132782
5. {whole wheat pasta}                         => {olive oil}     0.007998933
6. {herb & pepper,spaghetti}                   => {ground beef}   0.006399147
7. {herb & pepper,mineral water}               => {ground beef}   0.006665778
8. {tomato sauce}                              => {ground beef}   0.005332622
9. {mushroom cream sauce}                      => {escalope}      0.005732569
10. {frozen vegetables,mineral water,spaghetti} => {ground beef}   0.004399413

Как видно, в R apriori использовать на данный момент намного удобнее, чем в Python.

### ECLAT Algorithm

#### Теория

Идея алгоритма ECLAT заключается в ускорении подсчета $supp(X)$. Для этого нам необходимо проиндексировать наше базу данных $D$ так, чтобы это позволило быстро рассчитывать $supp(X)$

Легко заметить, что если $t(X)$ обозначает множество всех транзакций, где встречается подмножество $X$, то
<br>$t(XY) = t(X) \cap t(Y)$</br>
<br>и</br>
<br>$supp(XY) = |t(XY)|$</br>
<br> то есть $supp(XY)$ равен кардинальности (размеру) множества $t(XY)$

<img src="/src/images/tree_3.png">

Данный подход может быть значительно усовершенствован путем уменьшения размера промежуточных множеств идентификаторов транзакций (tidsets). А именно, мы можем хранить не все множество транзакций на промежуточном уровне, а только множество различий этих транзакций. Предположим, что
<br>$X_a = \{x_1, x_2,..., x_{n-1}, a\}$</br>
<br>$X_b = \{x_1, x_2,..., x_{n-1}, b\}$</br>
<br>Тогда, мы получим: </br>
<br>$X_{ab} = \{x_1, x_2,..., x_{n-1}, a, b\}$</br>
<br>$diffset(X_{ab})$ это множество всех id транзакций, которыесодержат префикс $X_a$ но не содержат элемент $b$: </br>
<br>$d(X_{ab}) =t(X_a)/t(X_{ab})=t(X_a)/t(X_{b})$</br>

<img src="/src/images/tree_4.png">

#### Реализация в Python

#### Реализация в R

И вновь пользователи R ликуют, для них никаких танцев с бубном делать не надо, все по аналогии с apriori. 

Запускаем библиотеку и читаем данные:

$library(arules)$
<br>
$dataset = read.csv('Market_Basket_Optimisation.csv')$
<br>
$dataset = read.transactions('Market_Basket_Optimisation.csv', sep = ',', rm.duplicates = TRUE)$


Быстрый взгляд на датасет:
<br>
$summary(dataset)$
<br>
$itemFrequencyPlot(dataset, topN = 10)$

Правила:
<br>
$rules = eclat(data = dataset, parameter = list(support = 0.003, minlen = 2))$ 
<br>
Обратите внимание, настраиваем толкьо support и минимальную длину (k в k-itemset)

И смотрим на результаты:
<br>
$inspect(sort(rules, by = 'support')[1:10])$

### FP-Growth Algorithm

#### Теория

#### Реализация в Python

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.mllib.fpm import FPGrowth
# $example off$
from pyspark import SparkContext

if __name__ == "__main__":
    sc = SparkContext(appName="FPGrowth")

    # $example on$
    data = sc.textFile("src/data/sample_fpgrowth.txt")
    transactions = data.map(lambda line: line.strip().split(' '))
    model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
    result = model.freqItemsets().collect()
    for fi in result:
        print(fi)
    # $example off$

#### Реализация в R

## Заключение